# Small scale proof of concept prototype for demo purposes

Github includes a few images that were used to test this prototype. The reason these arent continuously taken is in order to have the exact same environment for every picture. This is because the real testing station is not being built yet for this demo prototype.

In [1]:
# Imports
%matplotlib inline

import cv2
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image as image
import easygui

## Load a picture for classification
A new picture will be taken here in the real version

In [36]:
# Selecting image to classify
f = easygui.fileopenbox(filetypes=["*.jpg","*.jpeg","*.png", "*.bmp"])
bottle = cv2.imread(f)

In [37]:
# create mask of just the cap
# also mask of just he item
G = cv2.cvtColor(bottle, cv2.COLOR_BGR2GRAY)
ret, binaryCap = cv2.threshold(G, 80,255, cv2.THRESH_BINARY)
ret, binaryItem = cv2.threshold(G, 150,255, cv2.THRESH_BINARY)

bottleCopy = bottle.copy()

capFound = False
itemBig = False

In [38]:
# Find features (small rectangle of cap), done by extracting contours, 
# then finding a small area, this will most likely be the cap
contours, _ = cv2.findContours(binaryCap, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    currentArea = cv2.contourArea(contour)
    
    # condition that finds the cap
    if currentArea > 8000 and currentArea < 12000:
        (x, y, w, h) = cv2.boundingRect(contour) # generate coordinates for rectangle around object
    
        # Draw rectangle and a cap notation onto original, for demonstration purposes
        cv2.rectangle(bottleCopy, (x, y), (x + w, y + h), (0,0,255), 5)
        cv2.putText(bottleCopy,'Cap', (x,y-8), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255),2,cv2.LINE_AA)
        
        capFound = True

In [39]:
# Find rough length, if this is too small then it cant be the bottle
# extract contours from this binary image
binaryRev = cv2.bitwise_not(binaryItem)
contours, _ = cv2.findContours(binaryRev, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# get size of full image
height, width, d = bottle.shape
height = height * 0.6

for contour in contours:
    (x, y, w, h) = cv2.boundingRect(contour)
    
    #condition that makes sure the item is big enough to be a bottle
    if h > height:
        
        # Draw rectangle and a cap notation onto original, for demonstration purposes
        cv2.rectangle(bottleCopy, (x, y), (x + w, y + h), (0,255,0), 5)
        cv2.putText(bottleCopy,'Bottle', (x,y-8), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0),2,cv2.LINE_AA)
        
        itemBig = True

In [40]:
if capFound == True and itemBig == True:
    print("bottle found")
else:
    print("bottle not found")

bottle found


In [41]:
cv2.imshow("original", bottleCopy)
cv2.imshow("reverse", binaryRev)
cv2.imshow("itemMask", binaryItem)
cv2.imshow("CapMask", binaryCap)
key = cv2.waitKey(0) # the 0 here tells openCV to wait indefinitely (until a key is pressed)
cv2.destroyAllWindows()